## Simulating noise on MPQP

Quantum computers hold immense potential, but a major hurdle is ``noise``. ``Noise`` refers to anything that disrupts a quantum computer's calculations.

In this Notebook, we introduce the concept of ``noise`` in quantum computing. We'll take a practical approach to describing how noise affects qubits in a computation and show how you can build `Depolarizing` noise channel. 

### General imports
# Simulating noisy circuits

In this notebook we present how run noisy simulations by defining noise models and adding then to a ``QCircuit``.

If you are not familiar with the basic concepts of noise for quantum systems, we recommand you this introduction `TODO`

In [1]:
from mpqp import QCircuit
from mpqp.gates import *
from mpqp.noise import Depolarizing
from mpqp.execution import *

### Instantiate your noise model

In MPQP, the abstract class ``NoiseModel`` represents noisy channels acting on the qubits of the circuit, either after each gate application, or as an interaction with the environement (what is called idle noise). Each predefined noise model should extend this class, which has common attributes ``targets`` (indicating the indices of the qubits affected by this noise model) and the optional ``gates`` (indicating specific gates after which the noise will be applied)

For example, if one wants to apply a depolarizing noise on the circuit, he can use the class ``Depolarizing``. We must to specify as first argument the probability, or the error rate of the channel, and as second argument the qubits that will be targetted by this instance.

In [ ]:
Depolarizing(0.5, [0, 1, 2])

``dimension`` is by default equal to 1.

In [ ]:
Depolarizing(0.5, [0, 1, 2], dimension=2)
Depolarizing(0.5, [0, 1, 2], gates=[H, Rx, U])
Depolarizing(0.5, [0, 1, 2], dimension=2, gates=[CNOT, CZ])

### Adding noise to the circuit

In [ ]:
# using direct instantiation
circ = QCircuit([H(0), CNOT(0,1), Y(1), BasisMeasure([0,1], shots=100), Depolarizing(0.3, [0], gates=[H])])

# using add
circ = QCircuit([H(0), CNOT(0,1), Y(1), BasisMeasure([0,1], shots=100)])
circ.add([Depolarizing(0.3, [0]), Depolarizing(0.13, [1])])

# circuit.noises
print(circuit.noises)

# circuit.pretty_print()
circuit.pretty_print()

# circuit.without_noises()
print(circuit.without_noises)

# circuit.to_other_language(Language.BRAKET)
noisy_braket_circuit = circuit.to_other_language(Language.BRAKET)
print(noisy_braket_circuit)


In [2]:
# build your circuit
circuit = QCircuit(3, label="Noise-Simulation")
circuit.add([H(0), CNOT(0,1), T(2)])

## Types of Noise in Quantum Simulations

There are different noise types in quantum computing simulations but here we just focus on ``Depolarizing Noise``, which is a combination of bit-flip and phase-flip errors, causing the qubit to lose its quantum information.

#### Depolarizing noise implemented in MPQP
The Depolarizing class can be applied to one or more qubits specified by the ``targets`` list. Additionally, the ``dimension`` parameter allows targeting specific gates within a quantum circuit. This flexibility allows researchers to simulate noise in various scenarios.


In [3]:
# define a depolarizing noise channel
depolarizing_noise = Depolarizing(prob=0.4, targets=[0, 1])

In [4]:
# apply depolarizing noise to the circuit
circuit.add(depolarizing_noise)

In [6]:
# add measurements
circuit.add(BasisMeasure([0, 1, 2], shots=1024))

In [5]:
circuit.pretty_print()

QCircuit Noise-Simulation: Size (Qubits,Cbits) = (3, 2), Nb instructions = 4
     ┌───┐     ┌─┐   
q_0: ┤ H ├──■──┤M├───
     └───┘┌─┴─┐└╥┘┌─┐
q_1: ─────┤ X ├─╫─┤M├
     ┌───┐└───┘ ║ └╥┘
q_2: ┤ T ├──────╫──╫─
     └───┘      ║  ║ 
c: 2/═══════════╩══╩═
                0  1 


### Running on noisy simulators

In [ ]:
# device.is_noisy()

# which device is supported

# run as any other computation

# observable and sample jobs only

In [7]:
# select the local noise simulator
device = AWSDevice.BRAKET_LOCAL_SIMULATOR

In [8]:
# run the circuit on the local simulator
result = run(circuit, device)

This program uses OpenQASM language features that may not be supported on QPUs or on-demand simulators.



In [9]:
print(result)

Result: AWSDevice, BRAKET_LOCAL_SIMULATOR
Counts: [297, 0, 207, 0, 205, 0, 315, 0]
Probabilities: [0.29003906 0.         0.20214844 0.         0.20019531 0.  0.30761719 0.        ]
State: 110, Index: 6, Count: 315, Probability: 0.3076171875
State: 100, Index: 4, Count: 205, Probability: 0.2001953125
State: 010, Index: 2, Count: 207, Probability: 0.2021484375
State: 000, Index: 0, Count: 297, Probability: 0.2900390625
Error: None


